In [1]:
import spacy
import pandas as pd
import numpy as np
import neuralcoref
import ast
import random
from tqdm import tqdm_notebook

depeches = pd.read_csv("depeches.csv")
photos = pd.read_csv("photos.csv",encoding='latin-1')

c:\users\maison\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
c:\users\maison\appdata\local\programs\python\python36\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
C:\Users\Maison\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,15,17,23,24,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
nlp=spacy.load("en_core_web_md")  
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

In [4]:
nlp_depeches = [nlp(depeches.news2[i]) for i in tqdm_notebook(range(depeches.shape[0]))]

C:\Users\Maison\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [60]:
ents_dict = []
for document in nlp_depeches:
    dictionary = {}
    for ent in document.ents:
        dep_list = []
        start = ent.start
        end = ent.end
        for i in range(start,end):
            word = document[i]
            if(word.head.text not in ent.text and (word.dep_ == 'compound' or word.dep_ == 'conj')):
                while(word.dep_ == 'compound' or word.dep_ == 'conj'):
                    word = word.head
                dep_list.append(word.dep_)
            else:
                dep_list.append(word.dep_)
        if(ent.text in dictionary.keys()):
            dictionary[ent.text] += dep_list
        else:
            dictionary[ent.text] = dep_list
    ents_dict.append(dictionary)

In [69]:
ents_dict_str = [str(ents_dictionary) for ents_dictionary in ents_dict]
ents_dict_df = pd.DataFrame(ents_dict_str)
ents_dict_df.to_csv("ents_dict_depeches.csv")

In [109]:
def mentioned(ent, cluster):
    for mention in cluster.mentions:
        if(ent.text in mention.text):
            return(True)

In [112]:
corefs_dict = []
for document in nlp_depeches:
    dictionary = {}
    clusters = document._.coref_clusters
    for ent in document.ents:
        corefs_list = []
        for cluster in clusters:
            if(mentioned(ent,cluster)):
                for mention in cluster.mentions:
                    mention_deps = []
                    start = mention.start
                    end = mention.end
                    for i in range(start,end):
                        word = document[i]
                        if(word.head.text not in mention.text and (word.dep_ == 'compound' or word.dep_ == 'conj')):
                            while(word.dep_ == 'compound' or word.dep_ == 'conj'):
                                word = word.head
                            mention_deps.append(word.dep_)
                        else:
                            mention_deps.append(word.dep_)
                    corefs_list.append((mention.text, mention_deps))     
        if(ent.text not in dictionary.keys()):
            dictionary[ent.text] = corefs_list
    corefs_dict.append(dictionary)

In [116]:
corefs_dict_str = [str(corefs_dictionary) for corefs_dictionary in corefs_dict]
corefs_dict_df = pd.DataFrame(corefs_dict_str)
corefs_dict_df.to_csv("corefs_dict_depeches.csv")

In [120]:
labels_dict = []
for document in nlp_depeches:
    dictionary = {}
    for ent in document.ents:
        if(ent.text not in dictionary.keys()):
            dictionary[ent.text] = [ent.label_]
        else:
            dictionary[ent.text] += [ent.label_]
    labels_dict.append(dictionary)

In [121]:
labels_dict_str = [str(labels_dictionary) for labels_dictionary in labels_dict]
labels_dict_df = pd.DataFrame(labels_dict_str)
labels_dict_df.to_csv("labels_dict_depeches.csv")

In [126]:
final_dataframe = pd.DataFrame({'Labels' : labels_dict, 'Ents' : ents_dict, 'Corefs' : corefs_dict})
final_dataframe.to_pickle('final_dataframe.csv')

In [128]:
final_dataframe.head()

,Labels,Ents,Corefs
0,"{'Iranian': ['NORP'], '24': ['CARDINAL'], 'ISN...","{'Iranian': ['amod'], '24': ['nummod'], 'ISNA'...","{'Iranian': [], '24': [], 'ISNA': [], 'Wednesd..."
1,"{'Faf du Plessis': ['PERSON'], 'South Africa':...","{'Faf du Plessis': ['compound', 'compound', 'n...","{'Faf du Plessis': [], 'South Africa': [], 'th..."
2,"{'Malaysia': ['GPE', 'GPE'], 'Wednesday': ['DA...","{'Malaysia': ['nsubjpass', 'appos'], 'Wednesda...","{'Malaysia': [('Malaysia', ['nsubjpass']), ('M..."
3,"{'Frank Lampard': ['PERSON'], 'first': ['ORDIN...","{'Frank Lampard': ['compound', 'nsubj'], 'firs...","{'Frank Lampard': [('Frank Lampard', ['compoun..."
4,"{'each day': ['DATE'], 'China': ['GPE', 'GPE']...","{'each day': ['det', 'npadvmod'], 'China': ['p...","{'each day': [], 'China': [('China', ['pobj'])..."


In [127]:
test_dataframe = pd.read_pickle('final_dataframe.csv')
test_dataframe.head()

,Labels,Ents,Corefs
0,"{'Iranian': ['NORP'], '24': ['CARDINAL'], 'ISN...","{'Iranian': ['amod'], '24': ['nummod'], 'ISNA'...","{'Iranian': [], '24': [], 'ISNA': [], 'Wednesd..."
1,"{'Faf du Plessis': ['PERSON'], 'South Africa':...","{'Faf du Plessis': ['compound', 'compound', 'n...","{'Faf du Plessis': [], 'South Africa': [], 'th..."
2,"{'Malaysia': ['GPE', 'GPE'], 'Wednesday': ['DA...","{'Malaysia': ['nsubjpass', 'appos'], 'Wednesda...","{'Malaysia': [('Malaysia', ['nsubjpass']), ('M..."
3,"{'Frank Lampard': ['PERSON'], 'first': ['ORDIN...","{'Frank Lampard': ['compound', 'nsubj'], 'firs...","{'Frank Lampard': [('Frank Lampard', ['compoun..."
4,"{'each day': ['DATE'], 'China': ['GPE', 'GPE']...","{'each day': ['det', 'npadvmod'], 'China': ['p...","{'each day': [], 'China': [('China', ['pobj'])..."


In [130]:
final_dataframe.equals(test_dataframe)

True